In [1]:
# Importing necessary functions (wget, BeautifulSoup, Pandas, requests)
import requests
from bs4 import BeautifulSoup
import wget
import calendar
import pickle
import re
import os
import sys
import time
import json
import pandas as pd
from sqlite3 import dbapi2 as sq3
from collections import Counter
from pathlib import Path
import numpy as np
import copy
from matplotlib import pyplot as plt


from collections import OrderedDict
PATHSTART = '.'
pd.options.display.max_columns = None
#import logging

#logging.basicConfig(filename="myfile.txt",level=logging.DEBUG)

base_url = 'https://dumps.wikimedia.org/other/mediawiki_history/'
elog_dict = {}

In [2]:
# function to generate list of directories
def generate_dir_list(url, db): #change name
    temp_url = url + db + '/'
    index = requests.get(str(temp_url)).text
    soup_index = BeautifulSoup(index, 'html.parser')
    temp_list = [a['href'].strip('/') for a in soup_index.find_all('a') if a.has_attr('href') and a['href'] != '../']
    return temp_list

In [3]:
# generating base url
wikidb_list = generate_dir_list(base_url, '')
base_url = base_url + wikidb_list[1] +'/'

In [4]:
# necessary lists for processing
wikidbs_to_remove = ['donatewiki', 'wikimania2015wiki', 'wikimania2016wiki', 'wikimania2017wiki', 'testwiki', 
                     'test2wiki', 'testwikidatawiki', 'metawiki', 'mediawikiwiki', 'outreachwiki', 'incubatorwiki', 
                     'votewiki', 'loginwiki', 'sewikimedia', 'trwikimedia', 'uawikimedia', 'wbwikimedia' 
                     'amwikimedia', 'arwikimedia', 'bdwikimedia', 'bewikimedia', 'brwikimedia', 'cawikimedia', 
                     'cnwikimedia', 'cowikimedia', 'dkwikimedia', 'etwikimedia', 'fiwikimedia', 'hiwikimedia', 
                     'idwikimedia', 'maiwikimedia', 'mkwikimedia', 'nxwikimedia', 'nlwikimedia', 'nowikimedia', 
                     'nycwikimedia', 'nzwikimedia', 'plwikimedia', 'ptwikimedia', 'rswikimedia', 'ruwikimedia']

monthwise_wikidb_files = ['wikidatawiki', 'commonswiki', 'enwiki']

yearwise_wikidb_files = ['dewiki', 'frwiki', 'eswiki', 'itwiki', 'ruwiki', 'jawiki', 'viwiki', 'zhwiki', 'ptwiki', 
                  'enwiktionary', 'plwiki', 'nlwiki', 'svwiki', 'arwiki', 'shwiki', 'cebwiki', 'mgwiktionary',
                  'fawiki', 'frwiktionary', 'ukwiki', 'hewiki', 'kowiki', 'srwiki', 'trwiki', 'huwiki', 
                  'cawiki', 'nowiki', 'fiwiki', 'cswiki', 'idwiki', 'rowiki', 'enwikisource', 'frwikisource',
                  'ruwiktionary', 'dawiki', 'bgwiki', 'enwikinews', 'specieswiki', 'thwiki']

non_single_file_wikidb = wikidbs_to_remove + monthwise_wikidb_files + yearwise_wikidb_files

req_cols = ['wiki_db', 'event_entity', 'event_timestamp','event_user_text', 'event_user_registration_timestamp', 
            'revision_id']

In [5]:
# scraping url for column names 
wiki_page=requests.get('https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Edits/Mediawiki_history_dumps')
index = wiki_page.text
soup = BeautifulSoup(index, 'html.parser')
sch_tbl = soup.select("h2:has(#Schema_details) + table")
df = pd.read_html(str(sch_tbl))[0]

all_cols = df['Field name'].tolist()

In [6]:
# modification to the list
wikidb_list = generate_dir_list(base_url, '')

# list of wikis having single files
single_file_wikidb = [x for x in wikidb_list if x not in non_single_file_wikidb]
len(single_file_wikidb)

838

In [7]:
# reading the new users csv file
master_db = sq3.connect('master.db')
new_users_df = pd.read_sql_query("""SELECT * FROM newusers""", master_db)
print('connected to newusers table of master_db')
new_users_df

connected to newusers table of master_db


,user_name,cohort,campaign,cat,country
0,PjeterGj,2016,WLE,Images_from_Wiki_Loves_Earth_2016_in_Albania,Albania
1,Joana Balani,2016,WLE,Images_from_Wiki_Loves_Earth_2016_in_Albania,Albania
2,Juela Metushi,2016,WLE,Images_from_Wiki_Loves_Earth_2016_in_Albania,Albania
3,JuljanGjura,2016,WLE,Images_from_Wiki_Loves_Earth_2016_in_Albania,Albania
4,Jurgen Troka,2016,WLE,Images_from_Wiki_Loves_Earth_2016_in_Albania,Albania
...,...,...,...,...,...
146225,Scorpione 68,2014,WLM,Images_from_Wiki_Loves_Monuments_2014,with_no_country
146226,Scorpione1977,2014,WLM,Images_from_Wiki_Loves_Monuments_2014,with_no_country
146227,Scrant26,2014,WLM,Images_from_Wiki_Loves_Monuments_2014,with_no_country
146228,Tanline,2014,WLM,Images_from_Wiki_Loves_Monuments_2014,with_no_country


In [8]:
# list of new users through all the years of all the campaigns
new_users_list = list(new_users_df['user_name'])
len(new_users_list)

146230

In [ ]:
# function to delete a file
def del_file(file):
    if os.path.exists(file) == True:
        os.remove(file)
        print('\n' + file + ' is deleted')
    else:
        print('No such file found')

# Downloading and processing single file wiki databases

In this, we will make first make a list of file urls and download them batchwise. The size of the batches will be given before running the code along with number of iterations in a single execution. 
From every downloaded file, we take the data or list of users from users table and append them to alledits table  

In [ ]:
# function to download and process single wiki database files
def append_single_file_wikidb(wikis_list, status_file, errors_file, no_of_files, 
                       no_of_batches, users_list, table_name, db_object):
    
    exp_dict = {}
    status_key = 0
    
    # generating file urls for all the wikis in the given list
    if os.path.exists('singlewiki_fileurls.csv') == True:
        single_wiki_url_list = pd.read_csv('singlewiki_fileurls.csv')['single_wiki_url'].to_list()
    else:
        single_wiki_url_list = []
        for db in wikis_list:
            try:
                #print(db)
                file_name = generate_dir_list(base_url, db)
                #print(file_name)
                file_url = base_url + str(db) + '/' + file_name[0]
                #print(file_url)
                single_wiki_url_list.append(file_url)
            except Exception as err:
                exp_dict[db] = str(err)
                pass
            dict = {'single_wiki_url':single_wiki_url_list}
            temp_df = pd.DataFrame(dict) 
            temp_df.to_csv('singlewiki_fileurls.csv')
            del temp_df        

    list_len = len(single_wiki_url_list)
    #print(list_len)
    
    #checking for last processed batch
    try:
        with open(status_file, 'rb') as handle:
            status_key = pickle.load(handle)
        j = status_key
    except Exception as err:
        exp_dict[status_file] = str(err)
        pass    

    # checking if all wikis the are processed or not
    try:
        if list_len == j:
            print('Completed')
            del_file('singlewiki_fileurls.csv')
            raise StopExecution
    except:
        pass
    
    # creating batches of urls
    temp_list = [single_wiki_url_list[index:index + no_of_files] for index in 
                 range(0, len(single_wiki_url_list), no_of_files)]
    
    # making set of batches to download and process the files
    for j in range(status_key, status_key + no_of_batches):
        try:
            for item in temp_list[j]:
                print(item)
                try:
                    file_name = wget.download(item)
                except Exception as err:
                    exp_dict[item] = str(err)
                    pass
                try:
                    data = pd.read_csv(file_name, names = all_cols, compression='bz2', sep='\t', 
                                       low_memory=False, on_bad_lines = 'warn', chunksize=1000000)
                    df = pd.DataFrame(columns= req_cols)
                    for temp_df in data:
                        df = pd.concat([df, temp_df[req_cols]], ignore_index=True)
                        print(file_name + 'Stage:', df.index.max())
                    users_filter = df['event_user_text'].isin(users_list)
                    df = df[users_filter]
                    df.to_sql(table_name, db_object, if_exists='append', index=False)
                    del df
                except Exception as err:
                    exp_dict[file_name] = str(err)
                    pass
                try:
                    if os.path.exists(file_name) == True:
                        os.remove(file_name)
                except Exception as err:
                    file_loc = os.getcwd()
                    exp_dict[file_loc] = str(err)
                    pass
        except Exception as err:
            exp_dict[status_key] = str(err)
            pass
        status_key = j + 1
        with open(status_file, 'wb') as stat_file:
            pickle.dump(status_key, stat_file)
        stat_file.close()
    print('Batch completed')
    with open(errors_file, 'a+') as err_file:
         err_file.write(json.dumps(exp_dict))
    err_file.close()

In [ ]:
# running the function
append_single_file_wikidb(single_file_wikidb, no_of_files=15, status_file = 'single_status.pickle', 
                   errors_file = 'single_error_log.txt', no_of_batches=10, 
                   users_list=new_users_list, table_name="alledits", db_object=master_db)

# Downloading and processing wiki databases which are segregated yearwise

In this, we will make first make a list of file urls and download them batchwise. The size of the batches will be given before running the code along with number of iterations in a single execution. 
From every downloaded file, we take the data or list of users from users table and append them to alledits table  

In [ ]:
# get year of the wiki database file
def year_file(db_name, file_name):
    i = file_name.split(db_name)[1].strip('.-')
    year = re.compile(r'\d{4}').findall(i)[0]
    return int(year)

In [ ]:
# function to download and process yearwise segregated wiki database files
def append_year_wikis(wikis_list, year, status_file, errors_file, no_of_files, 
                       no_of_batches, users_list, table_name, db_object):
    
    exp_dict = {}
    status_key = 0
    
    # generating file urls for all the wikis in the given list
    if os.path.exists('yearwisewiki_fileurls.csv') == True:
        yearwise_wiki_url_list = pd.read_csv('yearwisewiki_fileurls.csv')['year_wiki_url'].to_list()
    else:
        yearwise_wiki_url_list = []
        for db in wikis_list:
            try:
                wiki_all_list = generate_dir_list(base_url, db)
            except Exception as err:
                exp_dict[db] = str(err)
                pass

            # checking if the file is within specified year 
            for i in wiki_all_list:
                file_year = year_file(db, i)
                try:
                    if file_year >= year and file_year < 2023:
                        file_url = base_url + db + '/' + i
                        yearwise_wiki_url_list.append(file_url)
                except Exception as err:
                    exp_dict[i] = str(err)
                    pass
                dict = {'year_wiki_url':yearwise_wiki_url_list}
                temp_df = pd.DataFrame(dict) 
                temp_df.to_csv('yearwisewiki_fileurls.csv')
                del temp_df

    list_len = len(yearwise_wiki_url_list)
    #print(list_len)
    
    #checking for last processed batch
    try:
        with open(status_file, 'rb') as handle:
            status_key = pickle.load(handle)
        j = status_key
    except Exception as err:
        exp_dict[status_file] = str(err)
        pass    

    # checking if all wikis the are processed or not
    try:
        if list_len == j:
            print('Completed')
            raise StopExecution
    except:
        pass
    
    # creating batches of urls
    temp_list = [yearwise_wiki_url_list[index:index + no_of_files] for index in 
                 range(0, len(yearwise_wiki_url_list), no_of_files)]
    
    # making set of batches to download and process the files
    for j in range(status_key, status_key + no_of_batches):
        try:
            for item in temp_list[j]:
                print(item)
                try:
                    file_name = wget.download(item), bar=None)
                except Exception as err:
                    exp_dict[item] = str(err)
                    pass
                try:
                    data = pd.read_csv(file_name, names = all_cols, compression='bz2', sep='\t', 
                                       low_memory=False, on_bad_lines = 'warn', chunksize=1000000)
                    df = pd.DataFrame(columns= req_cols)
                    for temp_df in data:
                        df = pd.concat([df, temp_df[req_cols]], ignore_index=True)
                        print(file_name + 'Stage:', df.index.max())
                    users_filter = df['event_user_text'].isin(users_list)
                    df = df[users_filter]
                    df.to_sql(table_name, db_object, if_exists='append', index=False)
                    del df
                except Exception as err:
                    exp_dict[file_name] = str(err)
                    pass
                try:
                    if os.path.exists(file_name) == True:
                        os.remove(file_name)
                except Exception as err:
                    file_loc = os.getcwd()
                    exp_dict[file_loc] = str(err)
                    pass
        except Exception as err:
            exp_dict[status_key] = str(err)
            pass
    status_key = j + 1
    with open(status_file, 'wb') as stat_file:
        pickle.dump(status_key, stat_file)
    stat_file.close()
    with open(errors_file, 'a+') as err_file:
         err_file.write(json.dumps(exp_dict))
    err_file.close()

In [ ]:
# running the function
append_year_wikis(yearwise_wikidb_files, year=2010, no_of_files=8, status_file = 'year_status.pickle', 
                   errors_file = 'error_log.txt', no_of_batches=5, users_list=new_users_list, 
                   table_name="alledits", db_object=master_db)

# Downloading and processing wiki databases which are segregated monthwise

In this, we will make first make a list of file urls and download them batchwise. The size of the batches will be given before running the code along with number of iterations in a single execution. 
We will first check 
From every downloaded file, we take the data or list of users from users table and append them to alledits table  

In [ ]:
# get month and year of wiki database file
def year_month_file(db_name, file_name):
    i = file_name.split(db_name)[1].strip('.-')
    year = re.compile(r'\d{4}').findall(i)[0]
    j = file_name.split(year)[1].strip('.-')
    month = re.compile(r'\d{2}').findall(j)[0]
    # month_name = calendar.month_name[int(month)]
    return int(year), int(month)

In [ ]:
# # function to download and process monthwise segregated wiki database files
def append_month_wikis(wikis_list, year, month, status_file, errors_file, no_of_files, 
                       no_of_batches, users_list, table_name, db_object):
    
    exp_dict = {}
    status_key = 0
    
    # generating file urls for all the wikis in the given list
    if os.path.exists('monthwisewiki_fileurls.csv') == True:
        monthwise_wiki_url_list = pd.read_csv('monthwisewiki_fileurls.csv')['month_wiki_url'].to_list()
    else:
        monthwise_wiki_url_list = []
        for db in wikis_list:
            try:
                wiki_all_list = generate_dir_list(base_url, db)
            except Exception as err:
                exp_dict[db] = str(err)
                pass

            # checking if the file is within specified year and month 
            for i in wiki_all_list:
                file_year, file_month = year_month_file(db, i)
                try:
                    if (file_year == year and file_month >= month):
                        file_url = base_url + db + '/' + i
                        monthwise_wiki_url_list.append(file_url)
                    elif (file_year >= year+1):
                        file_url = base_url + db + '/' + i
                        monthwise_wiki_url_list.append(file_url)
                except Exception as err:
                    exp_dict[i] = str(err)
                    pass
                dict = {'month_wiki_url':monthwise_wiki_url_list}
                temp_df = pd.DataFrame(dict) 
                temp_df.to_csv('monthwisewiki_fileurls.csv')
                del temp_df
                
    list_len = len(monthwise_wiki_url_list)
    #print(list_len)
    
    #checking for last processed batch
    try:
        with open(status_file, 'rb') as handle:
            status_key = pickle.load(handle)
        j = status_key
    except Exception as err:
        exp_dict[status_file] = str(err)
        pass    

    # checking if all wikis the are processed or not
    try:
        if list_len == j:
            print('Completed')
            raise StopExecution
    except:
        pass
    
    # creating batches of urls
    temp_list = [monthwise_wiki_url_list[index:index + no_of_files] for index in 
                 range(0, len(monthwise_wiki_url_list), no_of_files)]
    
    # making set of batches to download and process the files
    for j in range(status_key, status_key + no_of_batches):
        try:
            for item in temp_list[j]:
                print(item)
                try:
                    file_name = wget.download(item)
                except Exception as err:
                    exp_dict[item] = str(err)
                    pass
                try:
                    data = pd.read_csv(file_name, names = all_cols, compression='bz2', sep='\t', 
                                       low_memory=False, on_bad_lines = 'warn', chunksize=1000000)
                    df = pd.DataFrame(columns= req_cols)
                    for temp_df in data:
                        df = pd.concat([df, temp_df[req_cols]], ignore_index=True)
                        print(file_name + 'Stage:', df.index.max())
                    users_filter = df['event_user_text'].isin(users_list)
                    df = df[users_filter]
                    df.to_sql(table_name, db_object, if_exists='append', index=False)
                    del df
                except Exception as err:
                    exp_dict[file_name] = str(err)
                    pass
                try:
                    if os.path.exists(file_name) == True:
                        os.remove(file_name)
                except Exception as err:
                    file_loc = os.getcwd()
                    exp_dict[file_loc] = str(err)
                    pass
        except Exception as err:
            exp_dict[status_key] = str(err)
            pass
    status_key = j + 1
    with open(status_file, 'wb') as stat_file:
        pickle.dump(status_key, stat_file)
    stat_file.close()
    with open(errors_file, 'a+') as err_file:
         err_file.write(json.dumps(exp_dict))
    err_file.close()

In [ ]:
# running the function
append_month_wikis(monthwise_wikidb_files, year=2017, month=9, no_of_files=12, status_file = 'month_status.pickle', 
                   errors_file = 'month_error_log.txt', no_of_batches=1, 
                   users_list=new_users_list, table_name="alledits", db_object=master_db)